In [ ]:
df = pd.read_csv("/Users/sumitgaurav/Documents/Repositories/GenAI Project/classification-logs/training/dataset/synthetic_logs.csv")
df.head()

,timestamp,source,log_message,target_label
0,27/06/25 07:20,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert
3,12/07/25 00:24,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status
4,02/06/25 18:25,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status


In [74]:
df.source.unique()

<StringArray>
[      'ModernCRM', 'AnalyticsEngine',        'ModernHR',   'BillingSystem',
   'ThirdPartyAPI',       'LegacyCRM']
Length: 6, dtype: str

In [75]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['log_message'].tolist())
embeddings[:2]

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1285.64it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


array([[-1.02939703e-01,  3.35459337e-02, -2.20260508e-02,
         1.55098503e-03, -9.86916851e-03, -1.78956285e-01,
        -6.34410381e-02, -6.01762049e-02,  2.81108320e-02,
         5.99620081e-02, -1.72618385e-02,  1.43374118e-03,
        -1.49560049e-01,  3.15288105e-03, -5.66030778e-02,
         2.71685906e-02, -1.49890184e-02, -3.54038104e-02,
        -3.62936780e-02, -1.45410160e-02, -5.61498804e-03,
         8.75538513e-02,  4.55120578e-02,  2.50964370e-02,
         1.00187566e-02,  1.24266557e-02, -1.39923617e-01,
         7.68695921e-02,  3.14095281e-02, -4.15253639e-03,
         4.36902307e-02,  1.71250347e-02, -8.00950974e-02,
         5.74006215e-02,  1.89091638e-02,  8.55261534e-02,
         3.96398902e-02, -1.34371802e-01, -1.44367898e-03,
         3.06708645e-03,  1.76854104e-01,  4.44882922e-03,
        -1.69274919e-02,  2.24266555e-02, -4.35050316e-02,
         6.09036861e-03, -9.98167414e-03, -6.23972639e-02,
         1.07371677e-02, -6.04894478e-03, -7.14660510e-0

In [76]:
#perform DBSCAN clustering
dbscan = DBSCAN(eps = 0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

#add cluster labels to dataframe
df['cluster'] = clusters
df.head()

,timestamp,source,log_message,target_label,cluster
0,27/06/25 07:20,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2
3,12/07/25 00:24,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0
4,02/06/25 18:25,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0


In [77]:
# Group by cluster to inspect patterns
clusters = df.groupby('cluster')['log_message'].apply(list)
sorted_clusters = clusters.sort_values(key=lambda x: x.map(len), ascending=False)

In [78]:
print("Clustered Patterns:")
for cluster_id, messages in sorted_clusters.items():
    if len(messages) > 10:
        print(f"Cluster {cluster_id}:")
        for msg in messages[:5]:
            print(f"  {msg}")

Clustered Patterns:
Cluster 0:
  nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118
  nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180
  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791
  nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2

In [79]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None

In [80]:
classify_with_regex("User User694 logged OUT.")

'User Action'

In [81]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()].shape

(500, 6)

In [82]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex

,timestamp,source,log_message,target_label,cluster,regex_label
0,27/06/25 07:20,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,NaN
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1,NaN
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,NaN
3,12/07/25 00:24,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,NaN
4,02/06/25 18:25,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,NaN
...,...,...,...,...,...,...
2405,13/08/25 07:29,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,0,NaN
2406,01/11/25 05:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,7,NaN
2407,03/08/25 03:07,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,0,NaN
2408,11/11/25 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,1,NaN


In [83]:
# Print target_labels that have 5 or fewer rows in df_non_regex
rare = df_non_regex['target_label'].value_counts()
rare_labels = rare[rare <= 5]
print(rare_labels)

target_label
Workflow Error         4
Deprecation Warning    3
Name: count, dtype: int64


In [88]:
df_non_legacy = df_non_regex[df_non_regex.source != 'LegacyCRM']
df_non_legacy.source.unique()

<StringArray>
['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem', 'ThirdPartyAPI']
Length: 5, dtype: str

In [89]:
filtered_embeddings = model.encode(df['log_message'].tolist())
filtered_embeddings[:2]

array([[-1.02939703e-01,  3.35459337e-02, -2.20260508e-02,
         1.55098503e-03, -9.86916851e-03, -1.78956285e-01,
        -6.34410381e-02, -6.01762049e-02,  2.81108320e-02,
         5.99620081e-02, -1.72618385e-02,  1.43374118e-03,
        -1.49560049e-01,  3.15288105e-03, -5.66030778e-02,
         2.71685906e-02, -1.49890184e-02, -3.54038104e-02,
        -3.62936780e-02, -1.45410160e-02, -5.61498804e-03,
         8.75538513e-02,  4.55120578e-02,  2.50964370e-02,
         1.00187566e-02,  1.24266557e-02, -1.39923617e-01,
         7.68695921e-02,  3.14095281e-02, -4.15253639e-03,
         4.36902307e-02,  1.71250347e-02, -8.00950974e-02,
         5.74006215e-02,  1.89091638e-02,  8.55261534e-02,
         3.96398902e-02, -1.34371802e-01, -1.44367898e-03,
         3.06708645e-03,  1.76854104e-01,  4.44882922e-03,
        -1.69274919e-02,  2.24266555e-02, -4.35050316e-02,
         6.09036861e-03, -9.98167414e-03, -6.23972639e-02,
         1.07371677e-02, -6.04894478e-03, -7.14660510e-0

In [91]:
# Align X with df_non_legacy (embeddings indexed by original df rows)
X = filtered_embeddings[df_non_legacy.index]
y = df_non_legacy['target_label']

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Complement Naive Bayes (requires non-negative features)
from sklearn.naive_bayes import ComplementNB
from sklearn.preprocessing import MinMaxScaler

# Scale embeddings to [0, 1] (ComplementNB requires non-negative values)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

cnb_model = ComplementNB()
cnb_model.fit(X_train_scaled, y_train)

y_pred_cnb = cnb_model.predict(X_test_scaled)
print(f"ComplementNB Accuracy: {accuracy_score(y_test, y_pred_cnb):.4f}")

report = classification_report(y_test, y_pred_cnb)
print(report)

ValueError: Found input variables with inconsistent numbers of samples: [2410, 1903]